In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from re import sub
from decimal import Decimal
import json
import numpy as np
from sklearn.preprocessing import scale

In [12]:
# random select 50000 data from isbn.cvs
# ind = np.random.choice(isbn_data.index, 50000, replace=False)
# data = isbn_data.values[ind, :]
# mydata = pd.DataFrame(data)
# mydata.to_csv('sub_isbn.csv',cols=['ISBN'])

In [13]:
isbn_data = pd.read_csv('sub_isbn.csv')

In [14]:
r1 = open('result_full.json')
d1 = json.load(r1)
r2 = open('Output.json')
d2 = json.load(r2)

In [15]:
def remove(obj):
    new_dict = dict()
    for k,v in obj.items():
        if v['rank'] is None:
            continue
        else:
            new_dict[k]=v
    return new_dict

In [16]:
d_with_value = remove(d1)

# data cleaning and integration

In [17]:
list_keys = list()
l = list(d_with_value.keys())
for i in range(len(l)):
    list_keys.append(l[i])

In [18]:
data_knn = pd.DataFrame(np.zeros([17406, 11]))
for i in range(17406):
    data_knn.loc[i, 0] = list_keys[i]
    data_knn.loc[i, 1] = int(('').join((d_with_value[list_keys[i]]['Cutomers Reviews'].split()[0]).split(',')))
    data_knn.loc[i, 2] = d_with_value[list_keys[i]]['Ratings']['1']
    data_knn.loc[i, 3] = d_with_value[list_keys[i]]['Ratings']['2']
    data_knn.loc[i, 4] = d_with_value[list_keys[i]]['Ratings']['3']
    data_knn.loc[i, 5] = d_with_value[list_keys[i]]['Ratings']['4']
    data_knn.loc[i, 6] = d_with_value[list_keys[i]]['Ratings']['5']
    data_knn.loc[i, 7] = d_with_value[list_keys[i]]['Ratings']['average score']
    data_knn.loc[i, 8] = d_with_value[list_keys[i]]['category']
    data_knn.loc[i, 9] = d_with_value[list_keys[i]]['rank']
    data_knn.loc[i, 10] = d_with_value[list_keys[i]]['price']

In [19]:
data_knn.columns = ['ISBN', 'Customer reviews', 'Ratings-1', 'Ratings-2',
                    'Ratings-3', 'Ratings-4', 'Ratings-5', 'Average score', 'Category', 'Rank', 'Price']

In [20]:
data_knn_clean = data_knn

In [21]:
data_knn_clean
data_knn_clean.to_csv('data_knn_clean.csv')

In [22]:
# convert rank to integer number and re-rank it as 1,2,3,...
rank_list = list(data_knn['Rank'])
rank_int = list()
for i in range(17406):
    split = rank_list[i].split(',')
    rank_int.append(int(''.join(split)))
array = np.array(rank_int)
rank_new = array.argsort()+1
data_knn_clean['Rank'] = rank_new
# data_knn[data_knn['Rank']==34]

# convert price to float with 2 or 1 decimal places
price_list = list()
for i in range(17406):
    price_list.append(float(list(data_knn['Price'])[i]))
data_knn_clean['Price'] = price_list

In [23]:
# convert ratings to float number
r_1 = list()
r_2 = list()
r_3 = list()
r_4 = list()
r_5 = list()
for i in range(17406):
    r_1.append(float(list(data_knn['Ratings-1'])[i].strip('%')))
    r_2.append(float(list(data_knn['Ratings-2'])[i].strip('%')))
    r_3.append(float(list(data_knn['Ratings-3'])[i].strip('%')))
    r_4.append(float(list(data_knn['Ratings-4'])[i].strip('%')))
    r_5.append(float(list(data_knn['Ratings-5'])[i].strip('%')))

data_knn_clean['Ratings-1'] = r_1
data_knn_clean['Ratings-2'] = r_2
data_knn_clean['Ratings-3'] = r_3
data_knn_clean['Ratings-4'] = r_4
data_knn_clean['Ratings-5'] = r_5

In [24]:
knn_dummy = pd.get_dummies(data_knn_clean['Category'], drop_first=True)

In [25]:
df = data_knn_clean
df = df.drop('Category', 1)
data_knn_final = pd.concat([df,knn_dummy],axis=1,join_axes=[df.index])

In [26]:
# data_knn_final

Dataframe for K-NN calculation - data_knn_final saved as data_knn_final.csv

In [27]:
# extract all 17406 ISBN that has value
# pd.DataFrame(list(data_knn_final['ISBN'])).to_csv('isbn_with_value.csv',cols=['ISBN'])
# data_knn_final.to_csv('data_knn_final.csv')

K-NN model test

In [28]:
user = pd.read_csv('user_knn.csv')
user = user.drop('Book.Rating', 1)
user_1 = data_knn_final[data_knn_final['ISBN'].isin(user['ISBN'])]
data_knn = pd.merge(user, user_1, on='ISBN')
# train and test set
index = np.random.rand(len(data_knn)) < 0.80
train_x = data_knn[index].iloc[:,]  # eliminate ISBN in calculation
test_x = data_knn[~index].iloc[:,]
scaled_train = pd.DataFrame(scale(train_x.iloc[:, 2:]), index=train_x.index, columns=train_x.columns[2:])
scaled_test = pd.DataFrame(scale(test_x.iloc[:, 2:]), index=test_x.index, columns=test_x.columns[2:])

In [29]:
scaled_train = pd.DataFrame(scale(train_x.iloc[:, 2:]), index=train_x.index, columns=train_x.columns[2:])
scaled_test = pd.DataFrame(scale(test_x.iloc[:, 2:]), index=test_x.index, columns=test_x.columns[2:])
# scaled_train.columns
weights = [0.008641099,0.00217803,0.00217803,0.00217803,0.00217803,0.00217803,0.008641099,0.008641099,0.008641099,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727,0.027272727]
# assign weights to each columns
scaled_test = scaled_test.multiply(weights)
scaled_train = scaled_train.multiply(weights)
train_x_scaled = pd.concat([scaled_train, train_x.iloc[:,11:]],axis=1,join_axes=[scaled_train.index])
test_x_scaled = pd.concat([scaled_test, test_x.iloc[:,11:]],axis=1,join_axes=[scaled_test.index])
# for each data point in test_x, compared with all datapoints in train_x
d = np.zeros(shape=(len(train_x_scaled), len(test_x_scaled)))
for i in range(len(test_x_scaled)):
    for j in range(len(train_x_scaled)):
        d[j, i] = np.sum(abs(test_x_scaled.values[i, ] - train_x_scaled.values[j, ]))
distance_df = pd.DataFrame(d)
# distance_df.shape # 2189 * 658

In [30]:
# try different k and compare accuracy
def knn(k):

    train_pool = list()
    for t in range(0, len(test_x_scaled)):
        dist_sort = distance_df.iloc[:, t].sort_values(ascending=True)
        dist_sort_df = pd.DataFrame(dist_sort)
        # index of nearest k neighbors
        dist_index = dist_sort_df.iloc[0:k, 0].index.values
        train_pool.append(sorted(dist_index))
    
    train_pool = np.asarray(train_pool)

    pred_pool = list()
    for i in range(len(test_x)):
        user_id = train_x[train_x['ISBN'] == test_x['ISBN'].values[i]].values[:, 0]
        all_user = list()
        for j in range(len(user_id)):
            all_isbn = train_x[train_x['User.ID'] == user_id[j]].index
            all_user.extend(all_isbn)
        pred_pool.append(sorted(all_user))

    pred_pool = np.asarray(pred_pool)

    count = 0
    for item in range(len(pred_pool)):
        if set(train_pool[item]) & set(pred_pool[item]):
            count += 1
    return count/len(pred_pool)

In [31]:
# knn(10)

# Recommend by k-NN

In [32]:
def find_nn(input_isbn):
    user = pd.read_csv('all_user_match.csv').drop('Book.Rating', 1)  # 4475
    data_knn_final = pd.read_csv('data_knn_final.csv')  # 17406
    user_1 = data_knn_final[data_knn_final['ISBN'].isin(user['ISBN'])]
    d = pd.merge(user, user_1, on='ISBN')
    data_knn = d.drop(d.columns[2:6], axis=1)
    data_knn_scaled = pd.DataFrame(scale(data_knn.iloc[:, 2:7]), index=data_knn.index, columns=data_knn.columns[2:7])
    d_new = data_knn.drop(data_knn.columns[2:7],axis = 1)
    data_knn_1 = pd.concat([data_knn_scaled,d_new],axis=1,join_axes=[data_knn_scaled.index])
    
    if input_isbn not in list(data_knn['ISBN']): # for the data we have user info, we return k-nn results
        find_index = data_knn_final[data_knn_final['ISBN'] == input_isbn].index
        row_select = pd.DataFrame(data_knn_final.iloc[find_index].iloc[0,10:]).T
        category = row_select.columns[(row_select == 1).iloc[0]][0]
        all_same_cat = data_knn_final[data_knn_final[category] == 1]
        get_ave_score = all_same_cat.sort_values(by = 'Average score', ascending=False)
        index_list = list(get_ave_score.index[0:5])
        ISBN_list = list(get_ave_score.loc[index_list]['ISBN'])
        if input_isbn in ISBN_list:
            ISBN_list.remove(input_isbn)
        else:
            ISBN_list=ISBN_list[:4]
    else:
        test_x = (pd.DataFrame(data_knn_1[data_knn_1['ISBN'] == input_isbn].iloc[0,:]).T).drop(['User.ID','ISBN'],axis = 1)
        train_x = (data_knn_1[data_knn_1['ISBN'] != input_isbn].iloc[:,:]).drop_duplicates('ISBN')
        train_x_1 = train_x.drop(['User.ID','ISBN'],1)
        d = np.zeros(shape=(len(train_x_1), 1))
        for j in range(len(train_x_1)):
             d[j] = np.sum(abs(test_x.values - train_x_1.values[j, ]))
        distance_df = pd.DataFrame(d)
        distance_df.columns = ['distance']
        distance_sort = pd.DataFrame(distance_df.iloc[:, 0].sort_values(ascending=True))
        index_list = distance_sort.index[0:5]
        data_list = train_x_1.iloc[index_list]
        index_list_1 = data_list.index.values.tolist()
        ISBN_list = list(data_knn_1.iloc[index_list_1]['ISBN'])
        if input_isbn in ISBN_list:
            ISBN_list.remove(input_isbn)
        else:
            ISBN_list=ISBN_list[:4]
        
    return ISBN_list

In [33]:
input_isbn = '1572702516'
find_nn(input_isbn)

['1566192943', '1862911827', '1402201400', '157120153X']

In [34]:
# data_knn[data_knn['ISBN'] == '0743437772']
user = pd.read_csv('all_user_match.csv').drop('Book.Rating', 1)  # 4475
data_knn_final = pd.read_csv('data_knn_final.csv')  # 17406
user_1 = data_knn_final[data_knn_final['ISBN'].isin(user['ISBN'])]
d = pd.merge(user, user_1, on='ISBN')
data_knn = d.drop(d.columns[2:6], axis=1)
data_knn_scaled = pd.DataFrame(scale(data_knn.iloc[:, 2:7]), index=data_knn.index, columns=data_knn.columns[2:7])
d_new = data_knn.drop(data_knn.columns[2:7],axis = 1)
data_knn_1 = pd.concat([data_knn_scaled,d_new],axis=1,join_axes=[data_knn_scaled.index])

In [35]:
test_x = (pd.DataFrame(data_knn_1[data_knn_1['ISBN'] == input_isbn].iloc[0,:]).T).drop(['User.ID','ISBN'],axis = 1)

# Recommend by User ID

In [36]:
def find_books(user_id):
    from operator import itemgetter
    user_match = pd.read_csv('all_user_match.csv')
    user_match
    all_isbn = list(user_match[user_match['User.ID'] == user_id]['ISBN'])
    all_id = list()
    for i in range(len(all_isbn)):
        all_id.extend(list(user_match[user_match['ISBN'] == all_isbn[i]]['User.ID']))
    all_isbn_1 = list()
    for j in range(len(all_id)):
        all_isbn_1.extend(list(user_match[user_match['User.ID'] == all_id[j]]['ISBN']))
    unique_isbn = list(set(all_isbn_1))
    if len(unique_isbn) <= 4:
        return unique_isbn
    else:
        find_score = dict()
        order_isbn = list()
        for i in range(len(unique_isbn)):
            find_score[unique_isbn[i]] = (np.sum(user_match[user_match['ISBN'] == unique_isbn[i]]['Book.Rating']))/len(user_match[user_match['ISBN'] == unique_isbn[i]]['Book.Rating'])
        result = sorted(find_score.items(), key = itemgetter(1), reverse = True)
        for item in range(4):
            order_isbn.append(result[item][0])
        return order_isbn

In [14]:
find_books(3145)

['069811681X', '087842055X', '1557482357', '055321392X']

In [40]:
# user_match = pd.read_csv('all_user_match.csv')
# user_match

# Dataframe for wordcloud

In [326]:
data_wordcloud = pd.DataFrame(np.zeros([17406, 13]))
for i in range(17406):
    data_wordcloud.loc[i, 0] = list_keys[i]
    try:
        data_wordcloud.loc[i, 1] = d_with_value[list_keys[i]]['Introduction']
    except KeyError:
        data_wordcloud.loc[i, 1] = 'NA'
    try:
        data_wordcloud.loc[i, 2] = d_with_value[list_keys[i]]['Stars']['0']
    except:
        data_wordcloud.loc[i, 2] = 'NA'
    try:
        data_wordcloud.loc[i, 3] = d_with_value[list_keys[i]]['Stars']['1']
    except KeyError:
        data_wordcloud.loc[i, 3] = 'NA'
    try:
        data_wordcloud.loc[i, 4] = d_with_value[list_keys[i]]['Stars']['2']
    except KeyError:
        data_wordcloud.loc[i, 4] = 'NA'
    try:
        data_wordcloud.loc[i, 5] = d_with_value[list_keys[i]]['Stars']['3']
    except KeyError:
        data_wordcloud.loc[i, 5] = 'NA'
    try:
        data_wordcloud.loc[i, 6] = d_with_value[list_keys[i]]['Stars']['4']
    except KeyError:
        data_wordcloud.loc[i, 6] = 'NA'
    try:
        data_wordcloud.loc[i, 7] = d_with_value[list_keys[i]]['Stars']['5']
    except KeyError:
        data_wordcloud.loc[i, 7] = 'NA'
    try:
        data_wordcloud.loc[i, 8] = d_with_value[list_keys[i]]['Stars']['6']
    except KeyError:
        data_wordcloud.loc[i, 8] = 'NA'
    try:
        data_wordcloud.loc[i, 9] = d_with_value[list_keys[i]]['Stars']['7']
    except KeyError:
        data_wordcloud.loc[i, 9] = 'NA'
    try:
        data_wordcloud.loc[i, 10] = d_with_value[list_keys[i]]['author']
    except KeyError:
        data_wordcloud.loc[i, 10] = 'NA'
    try:
        data_wordcloud.loc[i, 11] = d_with_value[list_keys[i]]['category']
    except KeyError:
        data_wordcloud.loc[i, 11] = 'NA'
    try:
        data_wordcloud.loc[i, 12] = d_with_value[list_keys[i]]['tags']
    except KeyError:
        data_wordcloud.loc[i, 12] = 'NA'

In [37]:
# data_wordcloud.columns = ['ISBN', 'Introduction', 'Star-0', 'Star-1', 'Star-2', 'Star-3',
#                           'Star-4', 'Star-5', 'Star-6', 'Star-7', 'author', 'category', 'tags']
# # data_wordcloud

# END